In [5]:
import csv
import os
import cv2
from ultralytics import YOLO

In [6]:
annotations_csv = os.path.join(os.pardir, 'Annotations/annotations.csv')
videos_folder = os.path.join(os.pardir, 'Videos')

outputs_path = os.path.join(os.pardir, 'Annotations_Output')
if not os.path.exists(outputs_path):
    os.makedirs(outputs_path)

model = YOLO("yolov8n-face.pt")

In [7]:
with open(annotations_csv, newline='') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)
    next(reader)
    # Loop through each row in the CSV file
    for idx, row in enumerate(reader):
        print(idx, row)
        # Assuming each row has three values, you can save them into variables
        # Change the variable names and number according to your CSV structure
        video_tag, clip_id, label, frame_no, x, y, w, h, person_id = row
        video_folder_path = os.path.join(videos_folder, video_tag)
        video_path = os.path.join(video_folder_path, 'Complete.mp4')

        output_video_path = os.path.join(outputs_path, video_tag)
        if not os.path.exists(output_video_path):
            os.makedirs(output_video_path)

        cam = cv2.VideoCapture(video_path)
        if not cam.isOpened():
            print("Error: Couldn't open the video file.")
            exit()

        cam.set(cv2.CAP_PROP_POS_FRAMES, int(frame_no))

        ret, frame = cam.read()

        # Check if the frame is read successfully
        if not ret:
            print("Error: Couldn't read the frame.")
            exit()

        height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
        width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))

        x = int((float(x)/100) * width)
        y = int((float(y)/100) * height)
        w = int((float(w)/100) * width)
        h = int((float(h)/100) * height)

        text_x = 0
        text_y = h-20

        print(y)
        print(h)
        print(y+h)
        print(x)
        print(w)
        print(x+w)

        crop_img = frame[y:y + h, x:x + w]

        results = model(crop_img)
        boxes = results[0].boxes

        for box in boxes:
            top_left_x = int(box.xyxy.tolist()[0][0])
            top_left_y = int(box.xyxy.tolist()[0][1])
            bottom_right_x = int(box.xyxy.tolist()[0][2])
            bottom_right_y = int(box.xyxy.tolist()[0][3])

        print(top_left_x)
        print(top_left_y)
        print(bottom_right_x)
        print(bottom_right_y)

        cropped_img = crop_img[top_left_y:bottom_right_y, top_left_x:bottom_right_x]

        # Load the cascade
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Read the input image
        img = cv2.imread('suspected_face.jpg')
        
        # Convert into grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        
        # If faces are found, it returns the positions of detected faces as Rect(x,y,w,h).
        if len(faces) > 0:
            print("True - Human face is present.")
        else:
            print("False - Human face is not present.")

        # Wait for a key press and then close the image window
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # crop_img = frame[top_left_y:bottom_right_y, top_left_x:bottom_right_x]

        # cv2.rectangle(crop_img, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (0, 255, 0), 2)
        # # Write the string close to the bounding box
        # cv2.putText(crop_img, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0))

        # Save the result
        cv2.imwrite(output_video_path + "\\" + str(idx) + '.jpg', crop_img)

FileNotFoundError: [Errno 2] No such file or directory: '..\\Annotations/annotations.csv'